In [ ]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq ollama==0.3.3 --progress-bar off
%pip install -qqq pathlib --progress-bar off
%pip install -qqq pandas --progress-bar off
%pip install -qqq PyPDF2 --progress-bar off
%pip install -qqq ollama --progress-bar off


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import json
from enum import Enum
import PyPDF2
import ollama
from datetime import datetime as Date

MODEL = "llama3.1:8b-instruct-q8_0"
 

In [3]:
def extract_pdf_text(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join(page.extract_text() for page in reader.pages)
    return text

pdf_text = extract_pdf_text("sensors.pdf")
print("text extracted from pdf")

text extracted from pdf


In [4]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"
 
 
def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
    )
    return response["response"]

In [5]:

SUMMARIZE_PROMPT = """
As a LiDAR sensor expert, your task is to extract and categorize all the LiDAR sensor names mentioned in the following text. 

**Output Requirements**:
1. Provide a concise and structured list of LiDAR sensors.
2. Categorize the sensors based on their application, type, or any mentioned specifications (e.g., Automotive, Industrial, Surveying).
3. Include the brand or model name if mentioned.

**Formatting**:
- Strictly follow the JSON template below:
{{
    "categories": [
        {{
            "name": "Category Name",
            "sensors": [
                "Sensor 1",
                "Sensor 2"
            ]
        }}
    ],
    "metadata": {{
        "total_categories": <number>,
        "total_unique_sensors": <number>
    }}
}}

**Additional Instructions**:
- Only include a sensor category if found along with the identified list of values.
- If no sensors are found in the text, return an empty JSON: {{}}.
- Do not include any additional explanations or information.

<text>
{text}
</text>
"""

In [8]:
from collections import defaultdict

# Chunk size for processing
CHUNK_SIZE = 1000
chunks = [pdf_text[i:i+CHUNK_SIZE] for i in range(0, len(pdf_text), CHUNK_SIZE)]

# Placeholder for final results
categories_dict = defaultdict(set)

# Loop through chunks and get responses
for chunk in chunks:
    formatted_prompt = SUMMARIZE_PROMPT.format(text=chunk)
    response = call_model(formatted_prompt)  # Call your model with the prompt   

    try:
        # Try parsing the response as JSON
        response_json = json.loads(response)
        
        # Ensure that "categories" exists in the response and that it's a list
        if "categories" in response_json and isinstance(response_json["categories"], list):
            for category in response_json["categories"]:
                category_name = category.get("name", "Unidentified Category")  # Use a default if "name" is missing
                sensors = category.get("sensors", [])
                categories_dict[category_name].update(sensors)
        else:
            print(f"No 'categories' key found in response: {response_json}")

    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: Could not parse response: {response}")
        print(f"Error details: {e}")
        continue


JSONDecodeError: Could not parse response: {}
 
(Note: The text does not mention any LiDAR sensors.)
Error details: Extra data: line 3 column 1 (char 5)
JSONDecodeError: Could not parse response: {}
{
    "categories": [
        {
            "name": "Unknown Sensors",
            "sensors": [
                "LiDAR sensor"
            ]
        }
    ],
    "metadata": {
        "total_categories": 1,
        "total_unique_sensors": 1
    }
}
Error details: Extra data: line 2 column 1 (char 3)


In [ ]:
# Prepare the final JSON structure
categories_list = []
unique_sensors = set()

# Loop over the categories_dict to create the list of categories and their sensors
for category_name, sensors in categories_dict.items():
    categories_list.append({
        "name": category_name,
        "sensors": sorted(sensors)
    })
    unique_sensors.update(sensors)



Results saved to out/Extracted_Data_11.19.2024_222553.json
